## Load file

In [ ]:
#!pip install pdfplumber

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import pandas as pd
import pdfplumber
import warnings
warnings.filterwarnings('ignore')

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
downloaded = drive.CreateFile({'id':"1bXB_R5LIDlt-olTpmW8xzETXKFOQ56Rs"}) 
downloaded.GetContentFile('list_of_bc_hospitals.pdf')    

In [ ]:
page_idx = [i for i in range (0, 14)]
data_frames = []

for i in page_idx:
  pdf = pdfplumber.open("/content/list_of_bc_hospitals.pdf")
  table=pdf.pages[i].extract_table()
  df = pd.DataFrame(table[1::],columns=table[0])
  df = df.iloc[:, :2]
  data_frames.append(df)

data_frames = pd.concat(data_frames, ignore_index=True)
data_frames.columns = ['new_col1', 'new_col2']
data_frames

,new_col1,new_col2
0,ABBOTSFORD \n32900 Marshall Road \nV2S 0C2 \nP...,Abbotsford Regional Hospital \nand Cancer Cent...
1,ALERT BAY \n49 School Rd \nV0N 1A0 \nPh: (250)...,Cormorant Island Community \nHealth Centre \nV...
2,ASHCROFT \n700 Elm St \nBox 488 \nV0K 1A0 \nPh...,Ashcroft and District General \nHospital \nInt...
3,BELLA BELLA \n88 Waglisla St \nBella Bella \n...,R.W. Large Memorial Hospital \nVancouver Coast...
4,BELLA COOLA \n(Mackay St) \nBox 220 \nV0T 1C0 ...,Bella Coola General Hospital \nVancouver Coast...
...,...,...
93,VICTORIA \n1952 Bay Street \nV8R 1J8 \nPh: (2...,Royal Jubilee Hospital \nVancouver Island Heal...
94,VICTORIA \n63 Gorge Rd E \nV9A 1L2 \nPh: (250...,The Gorge Road Hospital \nVancouver Island Hea...
95,VICTORIA \n#1 Hospital Way \nV8Z 6R5 \nPh: (2...,Victoria General Hospital \nVancouver Island H...
96,WHITE ROCK \n15521 Russell Ave \nV4B 2R4 \nPh:...,Peace Arch District Hospital \nFraser Health A...


In [ ]:
#split the string in each cell
for i in range(data_frames.shape[0]): #iterate over rows
    for j in range(data_frames.shape[1]): #iterate over columns
      data_frames.iloc[i, j] = str(data_frames.iloc[i, j]).split('\n')

data_frames

,new_col1,new_col2
0,"[ABBOTSFORD , 32900 Marshall Road , V2S 0C2 , ...","[Abbotsford Regional Hospital , and Cancer Cen..."
1,"[ALERT BAY , 49 School Rd , V0N 1A0 , Ph: (250...","[Cormorant Island Community , Health Centre , ..."
2,"[ASHCROFT , 700 Elm St , Box 488 , V0K 1A0 , P...","[Ashcroft and District General , Hospital , In..."
3,"[BELLA BELLA , 88 Waglisla St , Bella Bella ,...","[R.W. Large Memorial Hospital , Vancouver Coas..."
4,"[BELLA COOLA , (Mackay St) , Box 220 , V0T 1C0...","[Bella Coola General Hospital , Vancouver Coas..."
...,...,...
93,"[VICTORIA , 1952 Bay Street , V8R 1J8 , Ph: (...","[Royal Jubilee Hospital , Vancouver Island Hea..."
94,"[VICTORIA , 63 Gorge Rd E , V9A 1L2 , Ph: (25...","[The Gorge Road Hospital , Vancouver Island He..."
95,"[VICTORIA , #1 Hospital Way , V8Z 6R5 , Ph: (...","[Victoria General Hospital , Vancouver Island ..."
96,"[WHITE ROCK , 15521 Russell Ave , V4B 2R4 , Ph...","[Peace Arch District Hospital , Fraser Health ..."


In [21]:
import re # regular expression

pd.set_option('display.max_rows', None)  # display all rows

d1 = pd.DataFrame(data_frames['new_col1'].tolist())
d1.iloc[39, 3] = d1.iloc[39, 4]

print(d1.iloc[61, 1])
d1.iloc[61, 0] += d1.iloc[61, 1]

for i in range(data_frames.shape[0]): #iterate over rows
    if (len(re.findall(r'[A-Z]{1}[0-9]{1}[A-Z]{1}\s*[0-9]{1}[A-Z]{1}[0-9]{1}', d1.iloc[i, 2])))== 0:
      d1.iloc[i, 2] = d1.iloc[i, 3]

d1 = d1.iloc[:, :3].drop(d1.columns[1], axis = 1)
d1.columns = ['city', 'postal_code']

d1

QUEEN CHARLOTTE  


,city,postal_code
0,ABBOTSFORD,V2S 0C2
1,ALERT BAY,V0N 1A0
2,ASHCROFT,V0K 1A0
3,BELLA BELLA,V0T 1Z0
4,BELLA COOLA,V0T 1C0
5,BURNABY,V5G 2X6
6,BURNS LAKE,V0J 1E0
7,CAMPBELL RIVER,V9W 3V1
8,CASTLEGAR,V1N 2H7
9,CHEMAINUS,V0R 1K1


In [42]:
pd. set_option('display.max_colwidth', -1)
d2 = pd.DataFrame(data_frames['new_col2'].tolist())

for i in range(data_frames.shape[0]): #iterate over rows
  if 'Hospital' not in d2.iloc[i, 0] and 'Centre' not in d2.iloc[i, 0]:
    d2.iloc[i, 0] += d2.iloc[i, 1]
    d2.iloc[i, 1] = ""
  elif ('and ' in d2.iloc[i, 1]) and ('Hospital' not in d2.iloc[i, 0] and 'Centre' not in d2.iloc[i, 0]):
    d2.iloc[i, 0] += d2.iloc[i, 1]
    d2.iloc[i, 1] = ""
  elif 'Centre' in d2.iloc[i, 1]:
    d2.iloc[i, 0] += d2.iloc[i, 1]
    d2.iloc[i, 1] = ""
  elif  "&" in d2.iloc[i, 1]:
    d2.iloc[i, 0] += d2.iloc[i, 1]
    d2.iloc[i, 1] = ""

  if "Authority" in d2.iloc[i, 2]:
    d2.iloc[i, 1] += d2.iloc[i, 2]
    d2.iloc[i, 2] = ""
  
  if d2.iloc[i, 3] is not None and "Authority" in d2.iloc[i, 3]:
    d2.iloc[i, 1] += d2.iloc[i, 2] + d2.iloc[i, 3]
    d2.iloc[i, 2] = ""
    d2.iloc[i, 3] = ""

#Manually fixed some of the details
d2.iloc[87, 0] += d2.iloc[87, 3]
d2.iloc[87, 1] = d2.iloc[87, 3] + d2.iloc[87, 4]
d2.iloc[14, 1] = "Valley Vancouver Island Health Authority"
d2.iloc[86, 0] = "Sunny Hill Health Centre for Children"
d2.iloc[86, 1] = "Provincial Health Services Authority"
d2.iloc[58, 0] = "The University Hospital of Northern British Columbia"
d2.iloc[58, 1] = "Northern Health Authority"
d2.iloc[68, 0] = "Sechelt Hospital/shíshálh	Hospital"
d2.iloc[68, 1] = "Vancouver Coastal Health Authority"
d2.iloc[61, 0] = "Haida Gwaii Hospital and Health Centre-Xaayda Gwaay	Ngaaysdll Naay"
d2.iloc[61, 1] = "Northern Health Authority"
d2.iloc[14, 0] = "North Island Hospital, Comox	Valley"
d2.iloc[14, 1] = "Vancouver Island Health Authority"
d2.iloc[92, 0] = "Queen Alexandra Centre for	Children's Health"
d2.iloc[92, 1] = "Vancouver Island Health Authority"

d2 = d2.iloc[:, :2]
d2.columns = ['hospital_name', 'health_authority']

d2[['city', 'postal_code']] = d1

#Final result
d2

,hospital_name,health_authority,city,postal_code
0,Abbotsford Regional Hospital and Cancer Centre,Fraser Health Authority,ABBOTSFORD,V2S 0C2
1,Cormorant Island Community Health Centre,Vancouver Island Health Authority,ALERT BAY,V0N 1A0
2,Ashcroft and District General Hospital,Interior Health Authority,ASHCROFT,V0K 1A0
3,R.W. Large Memorial Hospital,Vancouver Coastal Health Authority,BELLA BELLA,V0T 1Z0
4,Bella Coola General Hospital,Vancouver Coastal Health Authority,BELLA COOLA,V0T 1C0
5,Burnaby Hospital,Fraser Health Authority,BURNABY,V5G 2X6
6,Lakes District Hospital and Health Centre,Northern Health Authority,BURNS LAKE,V0J 1E0
7,"North Island Hospital, Campbell River & District",Vancouver Island Health Authority,CAMPBELL RIVER,V9W 3V1
8,Castlegar and District Community Health Centre,Interior Health Authority,CASTLEGAR,V1N 2H7
9,Chemainus Health Care Centre,Vancouver Island Health Authority,CHEMAINUS,V0R 1K1
